<a href="https://colab.research.google.com/github/hadwin-357/ML_models/blob/main/custom_implement_SVM_through_Lagrange.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementing dual problem of SVM (Lagrange primal problem):

## The following theory taken from https://towardsdatascience.com/support-vector-machines-learning-data-science-step-by-step-f2a569d90f76

The following explanation is about the binary classification but generalizes to more classes.

Let $X$ be the matrix of $n$ samples of the $p$ features. We want to separate the two classes of $y$ with an hyperplan (a straight line in 2D, that is $p=2$). The separation equation is:

$$ w^T x + b = 0, w \in \mathbb{R}^{p}, x \in \mathbb{R}^{p}, b \in \mathbb{R} $$

Given $x_0$ a point on the hyperplan, the signed distance of any point $x$ to the hyperplan is :
$$ \frac{w}{\Vert w \Vert} (x - x_0) = \frac{1}{\Vert w \Vert} (w^T x + b) $$

If $y$, such that $y \in \{-1, 1\}$, is the corresponding label of $x$, the (unsigned) distance is :
$$ \frac{y}{\Vert w \Vert} (w^T x + b) $$

This is the update quantity used by the Rosenblatt Perceptron.

The __Maximum margin separator__ is aiming at maximizing $M$ such that :
$$ \underset{w, b}{\max} M $$
__Subject to :__
- $y_i(x_i^T w + b) \ge M, i = 1..n$
- $\Vert w \Vert = 1$

$x_i$ and $y_i$ are samples of $x$ and $y$, a row of the matrix $X$ and the vector $y$.

However, we may change the condition on the norm of $w$ such that : $\Vert w \Vert = \frac 1M$

Leading to the equivalent statement of the maximum margin classifier :
$$ \min_{w, b} \frac 12 \Vert w \Vert^2 $$
__Subject to : $y_i(x_i^T w + b) \ge 1, i = 1..n$__

The corresponding Lagrange primal problem is :

$$\mathcal{L}_p(w, b, \alpha) = \frac 12 \Vert w \Vert^2 - \sum_{i=0}^n \alpha_i (y_i(x_i^T w + b) - 1)$$

__Subject to:__
- $\alpha_i \ge 0, i\in 1..n$

This shall be __minimized__ on $w$ and $b$, using the corresponding partial derivates equal to 0, we get :
$$\begin{align}
\sum_{i=0}^n \alpha_i y_i x_i &= w \\
\sum_{i=0}^n \alpha_i y_i &= 0
\end{align}$$

From $\mathcal{L}_p$, we get the (Wolfe) dual :
$$\begin{align}
\mathcal{L}_d (\alpha)
&= \sum_{i=0}^n \alpha_i - \frac 12 \sum_{i=0}^n \sum_{k=0}^n \alpha_i \alpha_k y_i y_k x_i^T x_k \\
&= \sum_{i=0}^n \alpha_i - \frac 12 \sum_{i=0}^n \sum_{k=0}^n \langle \alpha_i y_i x_i, \alpha_k y_k  x_k \rangle \\
\end{align}$$

__Subject to :__
- $\alpha_i \ge 0, i\in 1..n$
- $\sum_{i=0}^n \alpha_i y_i = 0$

Which is a concave problem that is __maximized__ using a solver.

Strong duality requires (KKT) [2, chap. 5.5]:
- $\alpha_i (y_i(x_i^T w + b) - 1) = 0,  \forall i \in 1..n$

Implying that :
- If $\alpha_i > 0$, then $y_i(x_i^T w + b) = 1$, meaning that $x_i$ is on one of the two hyperplans located at the margin distance from the separating hyperplan. $x_i$ is said to be a support vector
- If $y_i(x_i^T w + b) > 1$, the distance of $x_i$ to the hyperplan is larger than the margin.

$$\mathcal{L}_d = \sum_{i=0}^n \alpha_i - \frac 12 \sum_{i=0}^n \sum_{k=0}^n \alpha_i \alpha_k y_i y_k x_i^T x_k $$

__Subject to :__
- $\sum_{i=0}^n \alpha_i y_i = \langle \alpha, y \rangle = 0$
- $\alpha_i \ge 0, i\in 1..n$

The classifier is built on the scipy.optimize.minimum solver. The implementation is correct but inefficient as it is not taking into account for the sparsity of the $\alpha$ vector.

In [1]:
from scipy import optimize
class SVM_lagrange():
  def __init__(self):
        self.alpha = None
        self.w = None
        self.supportVectors = None

  def fit(self, X, y):
      N = len(y)
      # Gram matrix of (X.y)
      Xy = X * y[:, np.newaxis]
      GramXy = np.matmul(Xy, Xy.T)

      # Lagrange dual problem
      def Ld0(G, alpha):
          return alpha.sum() - 0.5 * alpha.dot(alpha.dot(G))

      # Partial derivate of Ld on alpha
      def Ld0dAlpha(G, alpha):
          return np.ones_like(alpha) - alpha.dot(G)

      # Constraints on alpha of the shape :
      # -  d - C*alpha  = 0
      # -  b - A*alpha >= 0
      A = -np.eye(N)
      b = np.zeros(N)
      constraints = ({'type': 'eq',   'fun': lambda a: np.dot(a, y), 'jac': lambda a: y},
                      {'type': 'ineq', 'fun': lambda a: b - np.dot(A, a), 'jac': lambda a: -A})

      # Maximize by minimizing the opposite
      optRes = optimize.minimize(fun=lambda a: -Ld0(GramXy, a),
                                  x0=np.ones(N),
                                  method='SLSQP',
                                  jac=lambda a: -Ld0dAlpha(GramXy, a),
                                  constraints=constraints)
      self.alpha = optRes.x
      self.w = np.sum((self.alpha[:, np.newaxis] * Xy), axis=0)
      epsilon = 1e-6
      self.supportVectors = X[self.alpha > epsilon]
      # Any support vector is at a distance of 1 to the separation plan
      # => use support vector #0 to compute the intercept, assume label is in {-1, 1}
      supportLabels = y[self.alpha > epsilon]
      self.intercept = supportLabels[0] - np.matmul(self.supportVectors[0].T, self.w)

  def predict(self, X):
      """ Predict y value in {-1, 1} """
      assert(self.w is not None)
      assert(self.w.shape[0] == X.shape[1])
      return 2 * (np.matmul(X, self.w) > 0) - 1

In [3]:
from timeit import default_timer as timer
def print_train_time(start: float, end: float):
    """Prints difference between start and end time.

    Args:
        start (float): Start time of computation (preferred in timeit format).
        end (float): End time of computation.
        device ([type], optional): Device that compute is running on. Defaults to None.

    Returns:
        float: time between start and end in seconds (higher is longer).
    """
    total_time = end - start
    print(f"Train time : {total_time:.3f} seconds")
    return total_time

In [5]:
from sklearn import datasets
import numpy as np
cancer = datasets.load_breast_cancer()
#convert label 0 to -1 for SVM
y= np.where(cancer['target']==0, -1, 1)

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
X_train, X_test, y_train, y_test = train_test_split(
        cancer['data'], y, test_size=0.2, random_state=42
    )

In [6]:
start_time=timer()
model_1 =SVM_lagrange()
model_1.fit(X_train,y_train)
y_pred =model_1.predict(X_test)

end_time =timer()

print(f'accuracy score: {accuracy_score(y_test, y_pred)}')
print(f"confusion matrix:{confusion_matrix(y_test,y_pred)}")
print_train_time(start_time, end_time)


accuracy score: 0.37719298245614036
confusion matrix:[[43  0]
 [71  0]]
Train time : 30.897 seconds


30.896577390000004

#Conclusion


1.  The predict seems quite wrong, further troubleshoot is needed
2.  Using inv of matrix has O(N^3) in computating complexity, which significantly slow down the program.